In [78]:
import pandas as pd
import numpy as np
from functools import reduce
import gender_detector
import gender_guesser.detector as gender
pd.set_option('display.max_columns', 500)

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
datas = []
for year in range(1970, 2010):
    datas.append(pd.read_csv('./crisp_data/CRISP_PRJ_C_FY'+str(year)+'.csv'))

In [4]:
datas[0]

,APPLICATION_ID,ACTIVITY_CODE,ADMIN_PHS_ORG_CODE,APPLICATION_CODE,FUNDED_APPL_CODE_DC,FY,GRANT_NUM,GRANT_NUM_STEM,SUBPROJECT_ID,IRG_CODE,...,PI_DEPARTMENT,PI_DEPT_CODE,PI_FIRST_NAME,PI_LAST_NAME,PI_MIDDLE_NAME,PROJ_PERIOD_START_DATE,PROJ_PERIOD_END_DATE,PROJECT_TITLE,SOURCE_CODE_DC,STATE_CODE
0,3591064,N01,AH,1,Y,1970,1N01AH004000-00070,1N01AH004000,NaN,NaN,...,NaN,NaN,HARRIET,TIEBEL,NaN,1969-08-18T00:00:00,1969-12-29T00:00:00,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,HISTORY,NaN
1,3591099,N01,AH,1,Y,1970,1N01AH004103-00070,1N01AH004103,NaN,NaN,...,NaN,NaN,GERALD,ELLINGSON,NaN,1970-02-25T00:00:00,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,HISTORY,NaN
2,3591110,N01,AH,1,Y,1970,1N01AH004158-00070,1N01AH004158,NaN,NaN,...,NaN,NaN,NaN,NONE,NaN,1970-06-29T00:00:00,NaN,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,HISTORY,NaN
3,3591112,N01,AH,1,Y,1970,1N01AH004172-00070,1N01AH004172,NaN,NaN,...,NaN,NaN,PHIL,WELSH,NaN,1970-06-30T00:00:00,1971-05-15T00:00:00,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,HISTORY,NaN
4,3591115,N01,AH,1,Y,1970,1N01AH004196-00070,1N01AH004196,NaN,NaN,...,NaN,NaN,RAYMOND,GIESLER,H,1970-06-29T00:00:00,1971-12-31T00:00:00,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HISTORY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,3672201,N01,NU,9,Y,1970,9N01NU004206-00070,9N01NU004206,NaN,NaN,...,NaN,NaN,WENDY,SIMON,NaN,1970-06-29T00:00:00,NaN,ENCOURAGE MINORITY YOUTH TO PURSUE A CAREER IN...,HISTORY,NaN
2133,3674669,N01,PM,9,Y,1970,9N01PM004149-00070,9N01PM004149,NaN,NaN,...,NaN,NaN,NORMAN,STEARNS,S,1970-04-20T00:00:00,NaN,EDUCATIONAL PROGRAM FOR PHYSICIANS IN DEPRESSE...,HISTORY,NaN
2134,3674670,N01,PM,9,Y,1970,9N01PM004150-00070,9N01PM004150,NaN,NaN,...,NaN,NaN,NORMAN,STEARNS,S,1970-04-20T00:00:00,NaN,DEV COMMUNITY HOSPITAL EDUCATIONAL PROGRAM FOR...,HISTORY,NaN
2135,3674672,N01,PM,9,Y,1970,9N01PM004189-00070,9N01PM004189,NaN,NaN,...,NaN,NaN,L,KIMBROUGH,S,1970-04-15T00:00:00,NaN,IMPLEMENT THE PROGRAM PHYSICIAN-IN-RESIDENCE,HISTORY,NaN


In [5]:
df = datas[0]
for i in range(len(datas)):
    if i != 0:
        df = df.append(datas[i])

In [6]:
cols_dropped = 'ADMIN_PHS_ORG_CODE,APPLICATION_CODE,FUNDED_APPL_CODE_DC,SUBPROJECT_ID,IRG_CODE,PROJ_PERIOD_START_DATE,PROJ_PERIOD_END_DATE,APPL_ID'.split(',')
all_cols = 'APPLICATION_ID,ACTIVITY_CODE,ADMIN_PHS_ORG_CODE,APPLICATION_CODE,FUNDED_APPL_CODE_DC,FY,GRANT_NUM,GRANT_NUM_STEM,SUBPROJECT_ID,IRG_CODE,MAJOR_ACTIVITY_CODE,ORG_NAME,PI_DEPARTMENT,PI_DEPT_CODE,PI_FIRST_NAME,PI_LAST_NAME,PI_MIDDLE_NAME,PROJ_PERIOD_START_DATE,PROJ_PERIOD_END_DATE,PROJECT_TITLE,SOURCE_CODE_DC,STATE_CODE,APPL_ID'.split(',')
cols_keep = list(set(all_cols)-set(cols_dropped))
cols_keep

['SOURCE_CODE_DC',
 'MAJOR_ACTIVITY_CODE',
 'PI_DEPARTMENT',
 'GRANT_NUM',
 'PROJECT_TITLE',
 'ORG_NAME',
 'PI_MIDDLE_NAME',
 'PI_DEPT_CODE',
 'FY',
 'PI_FIRST_NAME',
 'ACTIVITY_CODE',
 'GRANT_NUM_STEM',
 'STATE_CODE',
 'APPLICATION_ID',
 'PI_LAST_NAME']

In [7]:
df2 = df[cols_keep] # df is all the data, df2 is just the columns that we want
df2['ZIP_CODE'] = '-1'
df2['INDEX'] = [i for i in range(len(df2['ZIP_CODE']))]
df2.head()

<ipython-input-7-29c4c6939db1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ZIP_CODE'] = '-1'
<ipython-input-7-29c4c6939db1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['INDEX'] = [i for i in range(len(df2['ZIP_CODE']))]


,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,NaN,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,NaN,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NaN,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,NaN,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,NaN,3591115.0,GIESLER,-1,4


In [8]:
df2.shape

(2206074, 17)

##### df2['ORG_NAME'].value_counts()

In [9]:
df_dropped = df2.drop_duplicates(subset=["ORG_NAME"])
df_dropped = df_dropped[df_dropped['ORG_NAME'].notna()]

In [10]:
df_dropped

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,NaN,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,NaN,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NaN,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,NaN,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,NaN,3591115.0,GIESLER,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91849,CURRENT,N,NaN,N43EY000000-00009,NEXTGEN SERVER LICENSE - RENEWAL PERIOD: 2/1/...,"NEXTGEN HEALTHCARE INFORMATION SYS, INC.",NaN,NaN,2009,PATRICK,N43,N43EY000000,PA,7902324.0,CLINE,-1,2206065
91850,CURRENT,N,NaN,N43HB000000-00009,BIOMEDICAL (BASIC),"SCITECH STRATEGIES, INC.",NaN,NaN,2009,RICHARD,N43,N43HB000000,PA,7934751.0,KLAVANS,-1,2206066
91851,CURRENT,N,NaN,N43HC000000-00009,BIOMEDICAL (BASIC),"MAGIQ TECHNOLOGIES, INC.",NaN,NaN,2009,MICHAEL,N43,N43HC000000,MA,7937257.0,LAGASSE,-1,2206067
91854,CURRENT,N,NaN,N44CO000000-00009,DEVELOPMENT OF SOFTWARE SYSTEMS TO FACILITATE ...,QUANTUMMARK LLC,NaN,NaN,2009,SUZANNE,N44,N44CO000000,NV,7944352.0,MARCH,-1,2206070


In [11]:
df_dropped.shape

(20527, 17)

In [12]:
#df_dropped.to_csv('uniqueorgs.csv')

In [13]:
null_zips = df_dropped[df_dropped['STATE_CODE'].isnull()]
null_zips.head(6)

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,NaN,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,NaN,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NaN,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,NaN,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,NaN,3591115.0,GIESLER,-1,4
5,HISTORY,N,NaN,1N01AI002011-00070,TEST FOR HUMAN LEUKOCYTE TYPING,UNIVERSITY OF IOWA,S,NaN,1970,JOHN,N01,1N01AI002011,NaN,3591635.0,THOMPSON,-1,5


In [14]:
#null_zips.to_csv('nullzips.csv')
df_dropped[df_dropped['INDEX']==5]

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
5,HISTORY,N,NaN,1N01AI002011-00070,TEST FOR HUMAN LEUKOCYTE TYPING,UNIVERSITY OF IOWA,S,NaN,1970,JOHN,N01,1N01AI002011,NaN,3591635.0,THOMPSON,-1,5


In [15]:
i = 0
for index, row in null_zips.iterrows():
    print(row['INDEX'])
    getRowFilled = df2[df2['ORG_NAME']==row['ORG_NAME']]
    #print(getRowFilled)
    stateFilled = getRowFilled[getRowFilled['STATE_CODE'].notnull()]['STATE_CODE']
    print(row['ORG_NAME'])
    if(len(stateFilled)!=0):
        print(stateFilled.iloc[0])
        null_zips['STATE_CODE'][row['INDEX']] = stateFilled.iloc[0]
        #null_zips.iloc[row['INDEX']]['STATE_CODE'] = stateFilled.iloc[0]
        #null_zips.set_value(row['INDEX'], 'STATE_CODE', stateFilled.iloc[0])
    else:
        print(stateFilled)
    print('___')
    i += 1
print(i)

0
AMERICAN OCCUPATIONAL THERAPY ASSN
MD
___
1
U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC
Series([], Name: STATE_CODE, dtype: object)
___
2


<ipython-input-15-d1e3fdaad339>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_zips['STATE_CODE'][row['INDEX']] = stateFilled.iloc[0]
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


EDUCATIONAL TESTING SERVICE
NJ
___
3
COMPUTER APPLICATIONS, INC.
Series([], Name: STATE_CODE, dtype: object)
___
4


<ipython-input-15-d1e3fdaad339>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_zips['STATE_CODE'][row['INDEX']] = stateFilled.iloc[0]
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


HOSPITAL EDUCATIONAL AND RES FDN OF PA
PA
___
5
UNIVERSITY OF IOWA
IA
___
6


...

COMPANY FOR VACCINE & BIOL PRODTN NO, 1
Series([], Name: STATE_CODE, dtype: object)
___
1911771
UNIVERSIDAD DEL VALLE
Series([], Name: STATE_CODE, dtype: object)
___
1911954
HUMAN SCIENCES RESEARCH COUNCIL
Series([], Name: STATE_CODE, dtype: object)
___
1911961
KAGISO EDUCATIONAL TELEVISION
Series([], Name: STATE_CODE, dtype: object)
___
1911962
AIDS INFORMATION CENTRE - UGANDA
Series([], Name: STATE_CODE, dtype: object)
___
1911963
ACONDA
Series([], Name: STATE_CODE, dtype: object)
___
1911966
MALAWI BLOOD TRANSFUSION SERVICE
Series([], Name: STATE_CODE, dtype: object)
___
1911967
MINISTRY OF HEALTH AND PUBLIC HYGIENE
Series([], Name: STATE_CODE, dtype: object)
___
1911968
ALLIANCE NATIONALE CONTRE LE SIDA
Series([], Name: STATE_CODE, dtype: object)
___
1911969
EASTERN PROVINCIAL HEALTH OFFICE
Series([], Name: STATE_CODE, dtype: object)
___
1911970
NATIONAL REFERENCE LABORATORY
Series([], Name: STATE_CODE, dtype: object)
___
1911971
NATIONAL AIDS/STI CONTROL PROGRAM
Series([], Name: S

<ipython-input-15-d1e3fdaad339>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_zips['STATE_CODE'][row['INDEX']] = stateFilled.iloc[0]
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


INSERM LYON
Series([], Name: STATE_CODE, dtype: object)
___
1964528
HOSPITAL UNIV CLEMENTINO FRAGA FILHO
Series([], Name: STATE_CODE, dtype: object)
___
1964530
Universidade Federal do Rio de Janeiro
Series([], Name: STATE_CODE, dtype: object)
___
1965405
AMERICAN SOC. OF LAW, MEDICINE & ETHICS
Series([], Name: STATE_CODE, dtype: object)
___
1966575
FOUNDATION FOR REPRODUCTIVE HLH/FAM EDUC
Series([], Name: STATE_CODE, dtype: object)
___
1967385
CHONGQING UNIVERSITY OF MEDICAL SCIENCES
Series([], Name: STATE_CODE, dtype: object)
___
1967744
TOKYO UNIVERSITY OF EDUCATION
Series([], Name: STATE_CODE, dtype: object)
___
1967853
CENTER FOR GENOMIC REGULATION
Series([], Name: STATE_CODE, dtype: object)
___
1967885
ITALIAN NATIONAL CANCER INSTITUTE
Series([], Name: STATE_CODE, dtype: object)
___
1968041
MARTIN LUTHER UNIVERS-HALLE-WITTENBERG
Series([], Name: STATE_CODE, dtype: object)
___
1968256
UNIVERSITY OF VALPARAISO
Series([], Name: STATE_CODE, dtype: object)
___
1970141
UNIVERSITY OF WA

CHARITE, UNIVERSITAETSMEDIZIN BERLIN
Series([], Name: STATE_CODE, dtype: object)
___
2040845
SUN YAT-SEN UNIVERSITY
Series([], Name: STATE_CODE, dtype: object)
___
2041053
OKAIROS
Series([], Name: STATE_CODE, dtype: object)
___
2041063
NATIONAL CENTER /MATERNAL/ INFANT HLTH
Series([], Name: STATE_CODE, dtype: object)
___
2041088
ZAMBIA-EMORY HIV RESEARCH PROJECT
Series([], Name: STATE_CODE, dtype: object)
___
2042029
CANCER RESEARCH UK CAMBRIDGE RES INST
Series([], Name: STATE_CODE, dtype: object)
___
2042320
CHALMERS UNIVERSITY OF TECHNOLOGY
Series([], Name: STATE_CODE, dtype: object)
___
2042585
FACULTY OF MEDICINE OF TIMONE
Series([], Name: STATE_CODE, dtype: object)
___
2044824
SHANDONG PROVINCIAL CHEST HOSPITAL
Series([], Name: STATE_CODE, dtype: object)
___
2044827
INSTITUTE OF PRIMATE RESEARCH
Series([], Name: STATE_CODE, dtype: object)
___
2044828
NANJING MEDICAL UNIVERSITY
Series([], Name: STATE_CODE, dtype: object)
___
2045258
ROYAL COLLEGE OF SURGEONS IN IRELAND
Series([], N

INSTITUT PASTEUR DE MADAGASCAR
Series([], Name: STATE_CODE, dtype: object)
___
2131291
DIRECCION GEN DE VIGILANCIA DE LA SA
Series([], Name: STATE_CODE, dtype: object)
___
2131299
PATAN ACADEMY OF HEALTH SCIENCES
Series([], Name: STATE_CODE, dtype: object)
___
2131300
MINISTRY OF HEALTHCARE & NUTRITION
Series([], Name: STATE_CODE, dtype: object)
___
2135943
COLUMBIA UNIVERSITY OF NEW YORK
Series([], Name: STATE_CODE, dtype: object)
___
2164630
ARMED FORCES RESEARCH INST MED SCIENCES
Series([], Name: STATE_CODE, dtype: object)
___
2200176
JAMAICA MINISTRY OF HEALTH
Series([], Name: STATE_CODE, dtype: object)
___
2202116
SAINT GEORGE'S UNIVERSITY
Series([], Name: STATE_CODE, dtype: object)
___
2202485
UKZN INNOVATION
Series([], Name: STATE_CODE, dtype: object)
___
2202488
TRADITIONAL & MODERN HEALTH PRACTIONERS
Series([], Name: STATE_CODE, dtype: object)
___
2205228
BAOBAB HEALTH TRUST
Series([], Name: STATE_CODE, dtype: object)
___
2205240
KYUNG HEE UNIVERSITY
Series([], Name: STATE_COD

In [16]:
null_zips.head(10)

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,MD,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,NaN,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NJ,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,NaN,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,PA,3591115.0,GIESLER,-1,4
5,HISTORY,N,NaN,1N01AI002011-00070,TEST FOR HUMAN LEUKOCYTE TYPING,UNIVERSITY OF IOWA,S,NaN,1970,JOHN,N01,1N01AI002011,IA,3591635.0,THOMPSON,-1,5
6,HISTORY,N,NaN,1N01AI002029-00070,OPERATION OF A RUBELLA VIRUS SERVICE LABORATORY,NORTH AMERICAN BIOLOGICALS,R,NaN,1970,ROBERT,N01,1N01AI002029,NaN,3591640.0,RAFAJKO,-1,6
7,HISTORY,N,NaN,1N01AI002034-00070,MAINTAIN ANIMAL HOLDING FACILITY,MELPAR,B,NaN,1970,T,N01,1N01AI002034,NaN,3591643.0,OWEN,-1,7
8,HISTORY,N,MEDICINE,1N01AI002051-00070,SERVICE LABORATORY FOR NIH CHOLERA RESEARCH PR...,JOHNS HOPKINS UNIVERSITY,C,MDK,1970,CHARLES,N01,1N01AI002051,MD,3591644.0,CARPENTER,-1,8
9,HISTORY,N,MICROBIOLOGY,1N01AI002065-00070,STUDY ANTIGENICITY AND IMMUNOGENICITY OF M PNE...,WEILL MEDICAL COLLEGE OF CORNELL UNIV,B,MEK,1970,LAURENCE,N01,1N01AI002065,NY,3591664.0,SENTERFIT,-1,9


In [17]:
#null_zips[null_zips['STATE_CODE'].isnull()]
#null_zips[null_zips.index.duplicated()]
null_zips.index = null_zips['INDEX']

In [18]:
zips = pd.read_csv('./other_data/uszips.csv')
zips.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17242,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36073,-67.17517,Aguada,PR,Puerto Rico,True,NaN,38442,523.5,72003,Aguada,{'72003':100},Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,True,NaN,48814,667.9,72005,Aguadilla,{'72005':100},Aguadilla,72005,False,False,America/Puerto_Rico
3,606,18.16724,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6437,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29032,-67.12243,Anasco,PR,Puerto Rico,True,NaN,27073,312.0,72011,Añasco,"{'72003':0.55,'72011':99.45}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


In [19]:
for index, row in null_zips.iterrows():
    if(pd.isna(row['STATE_CODE'])):
        s = str(zips.sample()['state_id']).split('\n')[0]
        s = s[len(s)-2:]
        null_zips['STATE_CODE'][row['INDEX']] = s

<ipython-input-19-51934cb3a2c3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_zips['STATE_CODE'][row['INDEX']] = s
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
null_zips

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
INDEX,,,,,,,,,,,,,,,,,
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,MD,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,FL,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NJ,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,MO,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,PA,3591115.0,GIESLER,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202488,CURRENT,U,NaN,5U2GPS001088-02,SUPPORTING THE NATIONAL PREVENTION OF HIV MOTH...,TRADITIONAL & MODERN HEALTH PRACTIONERS,NaN,NaN,2009,HIZAAMU,U2G,5U2GPS001088,LA,7656819.0,RAMADHAN,-1,2202488
2205228,CURRENT,U,NaN,6U2GPS000740-03,IMPROVING THE DELIVERY & MANAGEMENT OF HIV /AI...,BAOBAB HEALTH TRUST,NaN,NaN,2009,SABINE,U2G,6U2GPS000740,KS,7678560.0,JOUKES,-1,2205228
2205240,CURRENT,F,NaN,7F05AT003770-0509,FMRI of Autonomic Regulation with Acupuncture,KYUNG HEE UNIVERSITY,NaN,NaN,2009,KYUNGMO,F05,7F05AT003770,TX,7945038.0,PARK,-1,2205240


In [21]:
df_dropped.index = df_dropped['INDEX']

In [22]:
for index, row in null_zips.iterrows():
    df_dropped['STATE_CODE'][row['INDEX']] = row['STATE_CODE']

<ipython-input-22-55bad84d4e6e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped['STATE_CODE'][row['INDEX']] = row['STATE_CODE']


In [23]:
df_dropped.head()

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
INDEX,,,,,,,,,,,,,,,,,
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,MD,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,FL,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NJ,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,MO,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,PA,3591115.0,GIESLER,-1,4


In [25]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
for index, row in df_dropped.iterrows():
    if row['STATE_CODE'] in states:
        getZip = zips[zips['state_id']==row['STATE_CODE']].sample()
        df_dropped['ZIP_CODE'][row['INDEX']] = getZip['zip'].item()

<ipython-input-25-a1b5b8095007>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped['ZIP_CODE'][row['INDEX']] = getZip['zip'].item()


In [26]:
df_dropped.head()

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
INDEX,,,,,,,,,,,,,,,,,
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,MD,3591064.0,TIEBEL,21638,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,FL,3591099.0,ELLINGSON,32463,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NJ,3591110.0,NONE,7820,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,MO,3591112.0,WELSH,65631,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,PA,3591115.0,GIESLER,16372,4


In [27]:
df3 = df2[df2['ORG_NAME'].notna()]
df3.index = df3['INDEX']
df_uniqueOrgs = df_dropped[['ORG_NAME','STATE_CODE', 'ZIP_CODE']]
df3.head()

,SOURCE_CODE_DC,MAJOR_ACTIVITY_CODE,PI_DEPARTMENT,GRANT_NUM,PROJECT_TITLE,ORG_NAME,PI_MIDDLE_NAME,PI_DEPT_CODE,FY,PI_FIRST_NAME,ACTIVITY_CODE,GRANT_NUM_STEM,STATE_CODE,APPLICATION_ID,PI_LAST_NAME,ZIP_CODE,INDEX
INDEX,,,,,,,,,,,,,,,,,
0,HISTORY,N,NaN,1N01AH004000-00070,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,AMERICAN OCCUPATIONAL THERAPY ASSN,NaN,NaN,1970,HARRIET,N01,1N01AH004000,NaN,3591064.0,TIEBEL,-1,0
1,HISTORY,N,NaN,1N01AH004103-00070,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,NaN,NaN,1970,GERALD,N01,1N01AH004103,NaN,3591099.0,ELLINGSON,-1,1
2,HISTORY,N,NaN,1N01AH004158-00070,DEVELOP EQUIVALENCY EXAMINATIONS IN MEDICAL TE...,EDUCATIONAL TESTING SERVICE,NaN,NaN,1970,NaN,N01,1N01AH004158,NaN,3591110.0,NONE,-1,2
3,HISTORY,N,NaN,1N01AH004172-00070,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,"COMPUTER APPLICATIONS, INC.",NaN,NaN,1970,PHIL,N01,1N01AH004172,NaN,3591112.0,WELSH,-1,3
4,HISTORY,N,NaN,1N01AH004196-00070,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,HOSPITAL EDUCATIONAL AND RES FDN OF PA,H,NaN,1970,RAYMOND,N01,1N01AH004196,NaN,3591115.0,GIESLER,-1,4


In [28]:
df3 = df3.set_index('ORG_NAME')
df_uniqueOrgs = df_uniqueOrgs.set_index('ORG_NAME')
df3.update(df_uniqueOrgs)
df3.reset_index(inplace=True)

In [29]:
cols = ['INDEX','FY','ORG_NAME','STATE_CODE','ZIP_CODE','PI_FIRST_NAME','PI_MIDDLE_NAME','PI_LAST_NAME','PI_DEPARTMENT','GRANT_NUM','GRANT_NUM_STEM','APPLICATION_ID','SOURCE_CODE_DC', 'ACTIVITY_CODE','MAJOR_ACTIVITY_CODE', 'PI_DEPT_CODE', 'PROJECT_TITLE']
df3 = df3[cols]

In [30]:
df4 = df3.dropna(subset=['PI_FIRST_NAME','PI_LAST_NAME'])

In [31]:
print(df4.shape)
df4.head()

(2073015, 17)


,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,GRANT_NUM_STEM,APPLICATION_ID,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE
0,0,1970,AMERICAN OCCUPATIONAL THERAPY ASSN,MD,21638,HARRIET,NaN,TIEBEL,NaN,1N01AH004000-00070,1N01AH004000,3591064.0,HISTORY,N01,N,NaN,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...
1,1,1970,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,FL,32463,GERALD,NaN,ELLINGSON,NaN,1N01AH004103-00070,1N01AH004103,3591099.0,HISTORY,N01,N,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...
3,3,1970,"COMPUTER APPLICATIONS, INC.",MO,65631,PHIL,NaN,WELSH,NaN,1N01AH004172-00070,1N01AH004172,3591112.0,HISTORY,N01,N,NaN,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...
4,4,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,1N01AH004196-00070,1N01AH004196,3591115.0,HISTORY,N01,N,NaN,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS
5,5,1970,UNIVERSITY OF IOWA,IA,51453,JOHN,S,THOMPSON,NaN,1N01AI002011-00070,1N01AI002011,3591635.0,HISTORY,N01,N,NaN,TEST FOR HUMAN LEUKOCYTE TYPING


In [32]:
df4[(df4['ORG_NAME']=='JOHNS HOPKINS UNIVERSITY') & (df4['PI_FIRST_NAME']=='CHARLES') & (df4['PI_LAST_NAME']=='CARPENTER')]


,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,GRANT_NUM_STEM,APPLICATION_ID,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE
8,8,1970,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,MEDICINE,1N01AI002051-00070,1N01AI002051,3591644.0,HISTORY,N01,N,MDK,SERVICE LABORATORY FOR NIH CHOLERA RESEARCH PR...
2453,2800,1971,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,MEDICINE,2N01AI002051-00270,2N01AI002051,3591646.0,HISTORY,N01,N,MDK,MAINTAIN SERVICE LABORATORY FOR NIH CHOLERA RE...
15609,15956,1972,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,MEDICINE,2N01AI002051-00470,2N01AI002051,3591648.0,HISTORY,N01,N,MDK,ANTITOXIC IMMUNITY IN CHOLERA
20874,21221,1972,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,MEDICINE,3N01AI002051-00570,3N01AI002051,3591649.0,HISTORY,N01,N,MDK,ADDITIONAL STUDIES OF ANTITOXIC IMMUNITY IN CH...
40596,40943,1972,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,NaN,5R22AI007625-06,5R22AI007625,4498819.0,HISTORY,R22,R,NaN,STUDIES IN AN EXPERIMENTAL MODEL OF CHOLERA
41104,41451,1972,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,NaN,5T01AI000009-14,5T01AI000009,4495290.0,HISTORY,T01,T,NaN,INFECTIOUS DISEASE AND CLINICAL IMMUNOLOGY


In [33]:
zipdrops = df4[df4['ZIP_CODE'] == '-1']
print(zipdrops.index)
df4 = df4.drop(zipdrops.index)

Int64Index([    472,     554,     568,     674,     784,     798,     901,
               1205,    1275,    1380,
            ...
            2088970, 2088971, 2089002, 2089053, 2089129, 2089464, 2089511,
            2089519, 2089643, 2089656],
           dtype='int64', length=8702)


In [34]:
df5 = df4.drop_duplicates(["ORG_NAME", "PI_FIRST_NAME", "PI_MIDDLE_NAME","PI_LAST_NAME"])

In [35]:
df5[(df5['ORG_NAME']=='JOHNS HOPKINS UNIVERSITY') & (df5['PI_FIRST_NAME']=='CHARLES') & (df5['PI_LAST_NAME']=='CARPENTER')]

,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,GRANT_NUM_STEM,APPLICATION_ID,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE
8,8,1970,JOHNS HOPKINS UNIVERSITY,MD,20839,CHARLES,C,CARPENTER,MEDICINE,1N01AI002051-00070,1N01AI002051,3591644.0,HISTORY,N01,N,MDK,SERVICE LABORATORY FOR NIH CHOLERA RESEARCH PR...


In [36]:
df5.shape

(301766, 17)

In [37]:
df5['INDEX'] = [i for i in range(len(df5['INDEX']))]
df5.index = df5['INDEX']
df5.head()

<ipython-input-37-014a7f799990>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['INDEX'] = [i for i in range(len(df5['INDEX']))]


,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,GRANT_NUM_STEM,APPLICATION_ID,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE
INDEX,,,,,,,,,,,,,,,,,
0,0,1970,AMERICAN OCCUPATIONAL THERAPY ASSN,MD,21638,HARRIET,NaN,TIEBEL,NaN,1N01AH004000-00070,1N01AH004000,3591064.0,HISTORY,N01,N,NaN,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...
1,1,1970,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,FL,32463,GERALD,NaN,ELLINGSON,NaN,1N01AH004103-00070,1N01AH004103,3591099.0,HISTORY,N01,N,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...
2,2,1970,"COMPUTER APPLICATIONS, INC.",MO,65631,PHIL,NaN,WELSH,NaN,1N01AH004172-00070,1N01AH004172,3591112.0,HISTORY,N01,N,NaN,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...
3,3,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,1N01AH004196-00070,1N01AH004196,3591115.0,HISTORY,N01,N,NaN,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS
4,4,1970,UNIVERSITY OF IOWA,IA,51453,JOHN,S,THOMPSON,NaN,1N01AI002011-00070,1N01AI002011,3591635.0,HISTORY,N01,N,NaN,TEST FOR HUMAN LEUKOCYTE TYPING


In [38]:
def truncated_normal(mean, stddev, minval, maxval, count):
        return np.clip(np.random.normal(mean, stddev, count), minval, maxval)

In [39]:
ages = truncated_normal(1980, 15, 1948, 2000, len(df5['INDEX']))
ages = [int(n) for n in ages]

months = np.random.uniform(1,13,len(df5['INDEX']))
months = [int(x) for x in months]

monthToYear = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
days = [1] * len(months)

for i in range(len(months)):
    numdays = monthToYear[months[i]]
    if months[i]==2:
        numdays = np.random.choice([28,28,28,29])
    days[i] = np.random.randint(1, numdays+1)
    

df5['YOB'] = ages
df5['MOB'] = months
df5['DOB'] = days

<ipython-input-39-f58fbe4ec699>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['YOB'] = ages
<ipython-input-39-f58fbe4ec699>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['MOB'] = months
<ipython-input-39-f58fbe4ec699>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [40]:
tmp = df5['PI_FIRST_NAME']
genders = [''] * len(tmp)
castGender = {'male':'M', 'female':'F', 'mostly_male':'M', 'mostly_female':'F', 'unknown':'unknown', 'andy':'unknown'}

genderModel = gender.Detector()

for i in range(len(tmp)):
    totest = tmp[i].lower()
    totest = totest[0:1].upper() + totest[1:]
    res = genderModel.get_gender(totest)
    res = castGender[res]
    if res=='unknown':
        randGender = np.random.randint(0,2)
        if randGender == 0:
            res = 'M'
        else:
            res = 'F'
    genders[i] = res

In [41]:
df5['SEX'] = genders
df5['SUBJECTHASMIDDLENAME'] = df5['PI_MIDDLE_NAME'].notnull()

<ipython-input-41-efe99e816efd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['SEX'] = genders
<ipython-input-41-efe99e816efd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['SUBJECTHASMIDDLENAME'] = df5['PI_MIDDLE_NAME'].notnull()


In [42]:
df5

,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,...,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE,YOB,MOB,DOB,SEX,SUBJECTHASMIDDLENAME
INDEX,,,,,,,,,,,,,,,,,,,,,
0,0,1970,AMERICAN OCCUPATIONAL THERAPY ASSN,MD,21638,HARRIET,NaN,TIEBEL,NaN,1N01AH004000-00070,...,HISTORY,N01,N,NaN,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,1999,7,3,F,False
1,1,1970,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,FL,32463,GERALD,NaN,ELLINGSON,NaN,1N01AH004103-00070,...,HISTORY,N01,N,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,1995,8,8,M,False
2,2,1970,"COMPUTER APPLICATIONS, INC.",MO,65631,PHIL,NaN,WELSH,NaN,1N01AH004172-00070,...,HISTORY,N01,N,NaN,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,1977,4,5,M,False
3,3,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,1N01AH004196-00070,...,HISTORY,N01,N,NaN,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,1959,3,29,M,True
4,4,1970,UNIVERSITY OF IOWA,IA,51453,JOHN,S,THOMPSON,NaN,1N01AI002011-00070,...,HISTORY,N01,N,NaN,TEST FOR HUMAN LEUKOCYTE TYPING,1984,3,16,M,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301761,301761,2009,"MAGIQ TECHNOLOGIES, INC.",MA,1775,MICHAEL,NaN,LAGASSE,NaN,N43HC000000-00009,...,CURRENT,N43,N,NaN,BIOMEDICAL (BASIC),1990,11,14,M,False
301762,301762,2009,"MAGARRAY, INC.",CA,92401,ROBERT,NaN,WHITE,NaN,N44CM000000-00109,...,CURRENT,N44,N,NaN,R&D-INSECT & DIS CONT-B RES,1958,4,6,M,False
301763,301763,2009,NEOCLONE BIOTECHNOLOGY INTERNATIONAL,WI,54513,UWE,NaN,MULLER,NaN,N44CN000000-00409,...,CURRENT,N44,N,NaN,Biomarker Screening with Revoluntionary Neoabs,1988,2,27,M,False


In [43]:
c = [''] * len(df5['ZIP_CODE'])
c[0] = 'a'

In [44]:
for i in range(len(df5['ZIP_CODE'])):
    getzip = df5['ZIP_CODE'][i]
    #print(zips.loc[zips['zip']==getzip, 'city'].item())
    getcity = zips.loc[zips['zip']==getzip, 'city']
    c[i] = getcity.item()

In [45]:
df5['COB'] = c

<ipython-input-45-8008a1065ba8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['COB'] = c


In [46]:
def make_ssn(s):
    def ssn_helper():
        first = str(np.random.randint(0,1000))
        middle = str(np.random.randint(0,100))
        last = str(np.random.randint(0,1000))
        if len(first) < 3:
            first = '0'*(3-len(first))+first
        if len(middle) < 2:
            middle = '0'*(2-len(middle))+middle
        if len(last) < 3:
            last = '0'*(3-len(last))+last
        return first+'-'+middle+'-'+last
    
    ssn = ssn_helper()
    while ssn in s:
        ssn = ssn_helper()
    return ssn

In [47]:
ssns = [''] * len(df5['INDEX'])
ssns_set = set()

for i in range(len(ssns)):
    get_ssn = make_ssn(ssns_set)
    ssns_set.add(get_ssn)
    ssns[i] = get_ssn

In [48]:
df5['GIID'] = ssns

<ipython-input-48-8cf0aec86e9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['GIID'] = ssns


In [49]:
df5['CNOB'] = 'US'

<ipython-input-49-bc8e30fcfc20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['CNOB'] = 'US'


In [50]:
df5.head()

,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,...,PI_DEPT_CODE,PROJECT_TITLE,YOB,MOB,DOB,SEX,SUBJECTHASMIDDLENAME,COB,GIID,CNOB
INDEX,,,,,,,,,,,,,,,,,,,,,
0,0,1970,AMERICAN OCCUPATIONAL THERAPY ASSN,MD,21638,HARRIET,NaN,TIEBEL,NaN,1N01AH004000-00070,...,NaN,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,1999,7,3,F,False,Grasonville,535-58-450,US
1,1,1970,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,FL,32463,GERALD,NaN,ELLINGSON,NaN,1N01AH004103-00070,...,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,1995,8,8,M,False,Wausau,014-49-941,US
2,2,1970,"COMPUTER APPLICATIONS, INC.",MO,65631,PHIL,NaN,WELSH,NaN,1N01AH004172-00070,...,NaN,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,1977,4,5,M,False,Clever,450-19-045,US
3,3,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,1N01AH004196-00070,...,NaN,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,1959,3,29,M,True,Clintonville,310-20-101,US
4,4,1970,UNIVERSITY OF IOWA,IA,51453,JOHN,S,THOMPSON,NaN,1N01AI002011-00070,...,NaN,TEST FOR HUMAN LEUKOCYTE TYPING,1984,3,16,M,True,Lohrville,956-64-372,US


In [51]:
cols = ['INDEX','SEX','PI_FIRST_NAME','PI_MIDDLE_NAME','PI_LAST_NAME','DOB', 'MOB', 'YOB','COB','ZIP_CODE','STATE_CODE','CNOB','SUBJECTHASMIDDLENAME','GIID','ORG_NAME','FY','GRANT_NUM','GRANT_NUM_STEM', 'PROJECT_TITLE']
df5 = df5[cols]
df5.head()

,INDEX,SEX,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,DOB,MOB,YOB,COB,ZIP_CODE,STATE_CODE,CNOB,SUBJECTHASMIDDLENAME,GIID,ORG_NAME,FY,GRANT_NUM,GRANT_NUM_STEM,PROJECT_TITLE
INDEX,,,,,,,,,,,,,,,,,,,
0,0,F,HARRIET,NaN,TIEBEL,3,7,1999,Grasonville,21638,MD,US,False,535-58-450,AMERICAN OCCUPATIONAL THERAPY ASSN,1970,1N01AH004000-00070,1N01AH004000,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...
1,1,M,GERALD,NaN,ELLINGSON,8,8,1995,Wausau,32463,FL,US,False,014-49-941,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,1970,1N01AH004103-00070,1N01AH004103,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...
2,2,M,PHIL,NaN,WELSH,5,4,1977,Clever,65631,MO,US,False,450-19-045,"COMPUTER APPLICATIONS, INC.",1970,1N01AH004172-00070,1N01AH004172,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...
3,3,M,RAYMOND,H,GIESLER,29,3,1959,Clintonville,16372,PA,US,True,310-20-101,HOSPITAL EDUCATIONAL AND RES FDN OF PA,1970,1N01AH004196-00070,1N01AH004196,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS
4,4,M,JOHN,S,THOMPSON,16,3,1984,Lohrville,51453,IA,US,True,956-64-372,UNIVERSITY OF IOWA,1970,1N01AI002011-00070,1N01AI002011,TEST FOR HUMAN LEUKOCYTE TYPING


In [52]:
#df5.to_csv('formatted_data_new.csv')

In [53]:
df5['GIID'].nunique()

301766

In [54]:
df5.shape

(301766, 19)

pred_census_ln(df5, 'PI_FIRST_NAME')

In [62]:
df6 = pd.merge(df4, df5, on = ['PI_FIRST_NAME', 'ORG_NAME', 'PI_MIDDLE_NAME', 'PI_LAST_NAME'], how = 'left')

In [63]:
df6.shape

(2064313, 33)

In [73]:
df6[df6['GIID']=='310-20-101'][['INDEX_x','INDEX_y']]

,INDEX_x,INDEX_y
3,4,3
359,389,3
1633,2033,3
2346,2797,3
3072,3531,3
3078,3537,3
15341,15952,3
20536,21197,3
20538,21200,3
21557,22251,3


In [76]:
cols = ['INDEX',
 'FY',
 'ORG_NAME',
 'STATE_CODE',
 'ZIP_CODE',
 'PI_FIRST_NAME',
 'PI_MIDDLE_NAME',
 'PI_LAST_NAME',
 'GRANT_NUM',
 'GRANT_NUM_STEM',
 'APPLICATION_ID',
 'PROJECT_TITLE',
 'SEX',
 'DOB',
 'MOB',
 'YOB',
 'COB',
 'CNOB',
 'SUBJECTHASMIDDLENAME',
 'GIID']

df6['INDEX'] = df6['INDEX_x']
df6['FY'] = df6['FY_x']
df6['STATE_CODE'] = df6['STATE_CODE_x']
df6['ZIP_CODE'] = df6['ZIP_CODE_x']
df6['GRANT_NUM'] = df6['GRANT_NUM_x']
df6['GRANT_NUM_STEM'] = df6['GRANT_NUM_STEM_x']
df6['PROJECT_TITLE'] = df6['PROJECT_TITLE_x']
df7 = df6[cols]

In [79]:
df7

,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,GRANT_NUM_STEM,APPLICATION_ID,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE,SEX,DOB,MOB,YOB,COB,CNOB,SUBJECTHASMIDDLENAME,GIID
0,0,1970,AMERICAN OCCUPATIONAL THERAPY ASSN,MD,21638,HARRIET,NaN,TIEBEL,NaN,1N01AH004000-00070,1N01AH004000,3591064.0,HISTORY,N01,N,NaN,ENCOURAGE INACTIVE THERAPISTS TO RE-ENTER PROF...,F,3,7,1999,Grasonville,US,False,535-58-450
1,1,1970,U.S. DEPT OF HLTH & HUMAN SRVS-DUMMY REC,FL,32463,GERALD,NaN,ELLINGSON,NaN,1N01AH004103-00070,1N01AH004103,3591099.0,HISTORY,N01,N,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,M,8,8,1995,Wausau,US,False,014-49-941
2,3,1970,"COMPUTER APPLICATIONS, INC.",MO,65631,PHIL,NaN,WELSH,NaN,1N01AH004172-00070,1N01AH004172,3591112.0,HISTORY,N01,N,NaN,SURVEY OF ALLIED HEALTH PROFESSIONS GRANTEE IN...,M,5,4,1977,Clever,US,False,450-19-045
3,4,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,1N01AH004196-00070,1N01AH004196,3591115.0,HISTORY,N01,N,NaN,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,M,29,3,1959,Clintonville,US,True,310-20-101
4,5,1970,UNIVERSITY OF IOWA,IA,51453,JOHN,S,THOMPSON,NaN,1N01AI002011-00070,1N01AI002011,3591635.0,HISTORY,N01,N,NaN,TEST FOR HUMAN LEUKOCYTE TYPING,M,16,3,1984,Lohrville,US,True,956-64-372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064308,2206069,2009,NEOCLONE BIOTECHNOLOGY INTERNATIONAL,WI,54513,UWE,NaN,MULLER,NaN,N44CN000000-00409,N44CN000000,7824215.0,CURRENT,N44,N,NaN,Biomarker Screening with Revoluntionary Neoabs,M,27,2,1988,Brantwood,US,False,508-09-848
2064309,2206070,2009,QUANTUMMARK LLC,NV,89820,SUZANNE,NaN,MARCH,NaN,N44CO000000-00009,N44CO000000,7944352.0,CURRENT,N44,N,NaN,DEVELOPMENT OF SOFTWARE SYSTEMS TO FACILITATE ...,F,7,1,1968,Battle Mountain,US,False,762-06-912
2064310,2206071,2009,"JGPERPICH, LLC",MD,20622,JOSEPH,NaN,PERPICH,NaN,N44DA000000-00409,N44DA000000,7902366.0,CURRENT,N44,N,NaN,NIDA International Virtual Collaboratory,M,18,11,1997,Charlotte Hall,US,False,518-42-667
2064311,2206072,2009,"PROLONG PHARMACEUTICALS, INC.",NJ,7718,ABE,NaN,ABUCHOWSKI,NaN,N44HB000000-00009,N44HB000000,7879547.0,CURRENT,N44,N,NaN,BIOMEDICAL (BASIC),M,23,2,1948,Belford,US,False,569-99-571


In [91]:
df7['INDEX'] = [i for i in range(len(df7['ZIP_CODE']))]


<ipython-input-91-9270a3ca9607>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['INDEX'] = [i for i in range(len(df7['ZIP_CODE']))]


In [92]:
df7[df7['GIID']=='310-20-101']

,INDEX,FY,ORG_NAME,STATE_CODE,ZIP_CODE,PI_FIRST_NAME,PI_MIDDLE_NAME,PI_LAST_NAME,PI_DEPARTMENT,GRANT_NUM,GRANT_NUM_STEM,APPLICATION_ID,SOURCE_CODE_DC,ACTIVITY_CODE,MAJOR_ACTIVITY_CODE,PI_DEPT_CODE,PROJECT_TITLE,SEX,DOB,MOB,YOB,COB,CNOB,SUBJECTHASMIDDLENAME,GIID
3,3,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,1N01AH004196-00070,1N01AH004196,3591115.0,HISTORY,N01,N,NaN,DEMONSTRATION WORK-STUDY FOR VIETNAM ERA VETERANS,M,29,3,1959,Clintonville,US,True,310-20-101
359,359,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,2N01AH004026-00170,2N01AH004026,3591066.0,HISTORY,N01,N,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,M,29,3,1959,Clintonville,US,True,310-20-101
1633,1633,1970,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,4N01AH004026-00070,4N01AH004026,3591065.0,HISTORY,N01,N,NaN,ESTABLISH AN OFFICE FOR HEALTH MANPOWER INFO &...,M,29,3,1959,Clintonville,US,True,310-20-101
2346,2346,1971,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,2N01AH004026-00370,2N01AH004026,3591068.0,HISTORY,N01,N,NaN,DEVELOP A STATE MANPOWER INTELLIGENCE FACILITY,M,29,3,1959,Clintonville,US,True,310-20-101
3072,3072,1971,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,3N01AH004026-00270,3N01AH004026,3591067.0,HISTORY,N01,N,NaN,CENTRAL STATE PROJECT FOR HEALTH MANPOWER INFO...,M,29,3,1959,Clintonville,US,True,310-20-101
3078,3078,1971,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,3N01AH004196-00170,3N01AH004196,3591116.0,HISTORY,N01,N,NaN,WORK-STUDY PROGRAM FOR VIETNAM ERA VETERANS,M,29,3,1959,Clintonville,US,True,310-20-101
15341,15341,1972,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,2N01AH004196-00370,2N01AH004196,3591118.0,HISTORY,N01,N,NaN,WORK-STUDY PROGRAM FOR VETERANS OF VIETNAM WAR,M,29,3,1959,Clintonville,US,True,310-20-101
20536,20536,1972,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,3N01AH004026-00470,3N01AH004026,3591069.0,HISTORY,N01,N,NaN,DEVELOP A STATE MANPOWER INTELLIGENCE FACILITY,M,29,3,1959,Clintonville,US,True,310-20-101
20538,20538,1972,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,3N01AH004196-00270,3N01AH004196,3591117.0,HISTORY,N01,N,NaN,WORK-STUDY PROGRAM FOR VIETNAM ERA VETERANS,M,29,3,1959,Clintonville,US,True,310-20-101
21557,21557,1972,HOSPITAL EDUCATIONAL AND RES FDN OF PA,PA,16372,RAYMOND,H,GIESLER,NaN,3N01MI004026-00572,3N01MI004026,3667877.0,HISTORY,N01,N,NaN,DEVELOP A STATE MANPOWER INTELLIGENCE FACILITY,M,29,3,1959,Clintonville,US,True,310-20-101


AT THIS POINT, DF7 IS THE COMPLETE DATASET WITH DUPLICATES AND INDEXED UNIQUELY ON 'GIID' AND DF5 IS BASICALLY DF7 WITHOUT DUPLICATES ON FIRST, MIDDLE, LAST NAME, AND ORGANIZATION

In [93]:
#df5.to_csv('dataset_complete_withoutduplicates.csv')

In [95]:
#df7.to_csv('dataset_complete.csv')